# Monsters Challange - 05 Spark Assessment

Import relevant modules:

In [63]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

In [64]:
# Create a SparkSession and context
spark = SparkSession.builder.appName("MonsterDataFrame").getOrCreate()

# Load data into dataframe
path = "./work/dnd_monsters.csv"
df = spark.read.options(header=True, inferSchema=True).csv(path)
print(df)
df.show()

DataFrame[name: string, url: string, cr: string, type: string, size: string, ac: int, hp: int, speed: string, align: string, legendary: string, source: string, str: double, dex: double, con: double, int: double, wis: double, cha: double]
+--------------------+--------------------+---+--------------------+------+---+---+---------+-------------+---------+--------------------+----+----+----+----+----+----+
|                name|                 url| cr|                type|  size| ac| hp|    speed|        align|legendary|              source| str| dex| con| int| wis| cha|
+--------------------+--------------------+---+--------------------+------+---+---+---------+-------------+---------+--------------------+----+----+----+----+----+----+
|           aarakocra|https://www.aided...|1/4|humanoid (aarakocra)|Medium| 12| 13|      fly| neutral good|     null| Monster Manual (BR)|10.0|14.0|10.0|11.0|12.0|11.0|
|             abjurer|                null|  9| humanoid (any race)|Medium| 12| 84|   

I used inferSchema to create a schema automatically rather than manually.

### SQL method

In [65]:
# Register the DataFrame as a temporary table
df.createOrReplaceTempView("monsters")

# Perform SQL queries on the DataFrame
sql_query = """
    SELECT 
        name, 
        CASE WHEN LOCATE('fly', speed) > 0 THEN dex - 2 ELSE dex END AS adjusted_dex
    FROM monsters
    WHERE legendary = 'Legendary'
    ORDER BY adjusted_dex DESC
    LIMIT 20
"""
result = spark.sql(sql_query)

# Show the query result
result.show()

+--------------------+------------+
|                name|adjusted_dex|
+--------------------+------------+
|              zariel|        22.0|
|               solar|        20.0|
|              moloch|        19.0|
|             vampire|        18.0|
|                lich|        16.0|
|             unicorn|        14.0|
|          gynosphinx|        13.0|
|  adult-black-dragon|        12.0|
|   adult-gold-dragon|        12.0|
|ancient-black-dragon|        12.0|
| ancient-gold-dragon|        12.0|
|               orcus|        12.0|
|              kraken|        11.0|
|           tarrasque|        11.0|
| adult-copper-dragon|        10.0|
|  adult-green-dragon|        10.0|
|ancient-copper-dr...|        10.0|
|ancient-green-dragon|        10.0|
|          mummy-lord|        10.0|
|             aboleth|         9.0|
+--------------------+------------+



### Pyspark method

In [66]:
# Create new dataframe with adjusted dex column
df_adjusted_dex = df.withColumn('adjusted_dex', when(df.speed.contains('fly'), df.dex - 2).otherwise(df.dex))

# Sort by new adjusted dex column and show top 20 which are legendary
df_adjusted_dex.select(df_adjusted_dex.name, df_adjusted_dex.adjusted_dex).where(df_adjusted_dex.legendary == 'Legendary').orderBy(col('adjusted_dex').desc()).show(20)

+--------------------+------------+
|                name|adjusted_dex|
+--------------------+------------+
|              zariel|        22.0|
|               solar|        20.0|
|              moloch|        19.0|
|             vampire|        18.0|
|                lich|        16.0|
|             unicorn|        14.0|
|          gynosphinx|        13.0|
|  adult-black-dragon|        12.0|
|   adult-gold-dragon|        12.0|
|ancient-black-dragon|        12.0|
| ancient-gold-dragon|        12.0|
|               orcus|        12.0|
|              kraken|        11.0|
|           tarrasque|        11.0|
| adult-copper-dragon|        10.0|
|  adult-green-dragon|        10.0|
|ancient-copper-dr...|        10.0|
|ancient-green-dragon|        10.0|
|          mummy-lord|        10.0|
|             aboleth|         9.0|
+--------------------+------------+
only showing top 20 rows

